# Day 2-Part 2: Reading and visualising data

We have already used the `numpy.loadtxt` function to load some data (the vertices of a polygon) from a text file. This function offers a lot of flexibility in the way we can read data from a text file by specifying options such as the data type (e.g., float or string), delimiter (the string used to separate values), skipping some lines (e.g., headers), using some columns, etc. You can find more about this in the [loadtxt documentation](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html). 

However, when working with more complex data such as data stored in spreadsheets or databases, the [`pandas`](https://pandas.pydata.org) (Python data analysis) library is the preferred choice. `pandas` will not only allow you to load and plot data, but as we will see throughout the course, it is great to explore and operate on data. Data analysis boils down to 🐼.

This notebook illustrates the use of `pandas` for reading and visualising data via two examples. Before we cover the examples, let's take a look at the core data structure of `pandas`, namely the `DataFrame`:

## DataFrame

A `DataFrame` is a two-dimensional object where data are stored in column and row format. Suppose we have the following table, and we want to make it into a `DataFrame`:

<img src="../figures/minerals.png" alt="minerals" width="400"/><br><br>

We can do that as follows:

In [ ]:
import pandas as pd # import pandas a pd

df = pd.DataFrame() # create an empty DataFrame

# add columns to the DataFrame using lists
# note that lists/columns should have the same length
df["mineral"] = ["halite", "quartz", "hematite", "rutile", "olivine"]
df["hardness"] = [2.5, 7, 6, 6, 7]
df["density"] = [2.17, 2.65, 5.3, 4.24, 3.4]

print(df) # view DataFrame

Here, we created the `DataFrame` using lists. We can also create the DataFrame using a `Dictionary`:

In [ ]:
# create Dictionary
my_dict = {"mineral":["halite", "quartz", "hematite", "rutile", "olivine"],
          "hardness":[2.5, 7, 6, 6, 7], "density":[2.17, 2.65, 5.3, 4.24, 3.4]}

# create DataFrame using Dictionary
df = pd.DataFrame(my_dict)

print(df) # view DataFrame

In this case the `Dictionary` keys are the columns, and the values are the entries of the columns. What type of object is a `DataFrame` column? Let's look at this:

In [ ]:
print(type(df["mineral"]), type(df["hardness"]), type(df["density"]))

The columns are of type `Series`. Series are more flexible than Lists, and as opposed to Lists, it is possible to operate on them, for example:

In [ ]:
df["hardness/density"] = df["hardness"] / df["density"] # add a column of hardness/density
print(df) # print DataFrame

We will use this powerful functionality throughout the course. Now that we know what a `DataFrame` is, we can use it to read and visualize data in the following two examples:

## Example 1: Worldwide earthquakes

The Excel file `eq_stat.xlsx` in the `data` directory includes the M > 5 earthquakes worldwide from the [USGS earthquake statistics](https://www.usgs.gov/programs/earthquake-hazards/lists-maps-and-statistics) site. Let's analyse these data using `pandas`.

First, we need to load the data. This is as easy as reading the file using the `pandas.read_excel` function. This loads the table as a `DataFrame` object, a two-dimensional labeled data structure with different columns. We can use the `DataFrame.head` method to display the first 5 (this is the default) rows of the table:

In [ ]:
import os # import operating system module

path = os.path.join("..", "data", "eq_stat.xlsx") # a safe path to the file
eq_stat = pd.read_excel(path) # read earthquakes statistics
eq_stat.head()

The leftmost bold integers are the indexes of the rows. Every row in a dataframe has an index, and in this case the indexes of the rows happen to be integers.

The first column are the ids of the rows (earthquake magnitudes and deaths), and the other columns are these data over the years 2000 to 2021. However, rather than having this information on a row basis, we would like to have it on a column basis. Let's use `pandas`to format the data as we want. We start by removing the 'Magnitude' column using the `DataFrame.drop` method:

In [ ]:
eq_stat = eq_stat.drop(columns=["Magnitude"]) # remove Magnitude column
eq_stat.head()

Then we make the rows into columns using the `DataFrame.transpose` method:

In [ ]:
eq_stat = eq_stat.transpose() # make the rows into columns
eq_stat.head()

Notice that now the indexes of the rows are the years. Finally, we reorganize the columns such that the lowest magnitude earthquakes (5-5.9) are in the first column, followed by the higher magnitude earthquakes in the next columns. In the last column, we still keep the deaths. We also rename the columns to the original id descriptions, magnitudes and deaths, using the `DataFrame.rename` method:

In [ ]:
new_cols = [3, 2, 1, 0, 4] # make a list of new columns
eq_stat = eq_stat[new_cols] # reorganize columns
eq_stat = eq_stat.rename(columns= {3:"5-5.9", 2:"6-6.9", 1:"7-7.9", 0:"8.0+", 4:"Deaths"}) # rename columns
eq_stat.head()

And we can also use the `DataFrame.tail` method to see the last 5 (this is the default) rows of the table:

In [ ]:
eq_stat.tail()

Notice that for the last two years, there is no information about Deaths. 

We are now ready to plot these data. We can use the `matplot.pyplot` module to do so. In the `plot` calls, the `DataFrame.iloc` function allows us to extract the desired entries by row and column indexes (for example, `iloc[:,0]` is the first column):

In [ ]:
import matplotlib.pyplot as plt # import pyplot module

fig, ax = plt.subplots()

ax.plot(eq_stat.iloc[:,0], ".-", color="dodgerblue", label=eq_stat.columns[0]) # plot 5-5.9 earthquakes
ax.plot(eq_stat.iloc[:,1], "g.-", label=eq_stat.columns[1]) # plot 6-6.9 earthquakes
ax.plot(eq_stat.iloc[:,2], ".-", color="orange", label=eq_stat.columns[2]) # plot 7-7.9 earthquakes
ax.plot(eq_stat.iloc[:,3], "r.-", label=eq_stat.columns[3]) # plot 8.0+ earthquakes

ax.set_xlabel("year") # set x label
ax.set_ylabel("number of earthquakes") # set y label
ax.set_xlim([2000, 2021]) # set x limits
my_xticks = [2000, 2005, 2010, 2015, 2020] # make a list of x ticks
ax.set_xticks(my_xticks) # set x ticks
ax.set_ylim([0, 3000]) # set y limits
ax.legend(); # include legend

This is fine. However, we can plot these data in a much simpler way using the `DataFrame.plot` function. This is a wrapper of the `pyplot.plot` function in `pandas`. To do that, we first make a new DataFrame without the Deaths column. Then, we use this DataFrame to plot the earthquakes in just one line using the `DataFrame.plot` function. Notice that in this line we retrieve an axis object (`ax`) to latter set the properties of the plot:

In [ ]:
eq_stat_mod = eq_stat.drop(columns=["Deaths"]) # new DataFrame without Deaths column

# plot earthquakes
ax = eq_stat_mod.plot(color={"5-5.9":"dodgerblue", "6-6.9":"g", "7-7.9":"orange", "8.0+":"r"}, style=".-")

ax.set_xlabel("year") # set x label
ax.set_ylabel("number of earthquakes") # set y label
ax.set_xlim([0, 21]) # set x limits
ax.set_ylim([0, 3000]); # set y limits, the legend is included by default

This plot is okay but is not exactly what we want. It would be better to vertically stack the earthquakes in a chart area graph, to clearly display the variation of earthquakes through time. Conveniently, the `DataFrame.plot.area` function does that:

In [ ]:
# plot earthquakes chart area
ax = eq_stat_mod.plot.area(color={"5-5.9":"dodgerblue", "6-6.9":"g", "7-7.9":"orange", "8.0+":"r"}, 
                           legend="reverse")

ax.set_xlabel("year") # set x label
ax.set_ylabel("number of earthquakes") # set y label
ax.set_xlim([0, 21]) # set x limits
ax.set_ylim([0, 3000]); # set y limits
ax.set_title("Worlwide earthquakes"); # set title of the plot

This looks much better. Let's add to this graph the number of earthquake related deaths. We can do this by adding another axis to the graph using the `ax.twinx` function. We then use this new axis to plot the Deaths column of the original `eq_stat` DataFrame. Notice that this time we use the `DataFrame.loc` function to retrieve the entries by their rows and columns labels. Also, we don't include the last two years of the `Deaths` column, because there are no entries for those years:

In [ ]:
# import numpy as np
import numpy as np

# plot earthquakes chart area and set graph properties
ax1 = eq_stat_mod.plot.area(color={"5-5.9":"dodgerblue", "6-6.9":"g", "7-7.9":"orange", "8.0+":"r"}, 
                            xlabel="year", ylabel="number of earthquakes", xlim=([0, 21]), 
                            ylim=([0, 3000]), legend="reverse", 
                            title="Worlwide earthquakes and related deaths")

ax2 = ax1.twinx() # add new axis sharing the x axis
ax2.plot(np.arange(0,20,1),eq_stat.loc[2000:2019,"Deaths"],"ko-") # plot Deaths on new axis
ax2.set_ylabel(eq_stat.columns[4]);

As you probably suspected, there is no correlation whatsoever between the number of earthquakes and the number of deaths in a year. Seismic hazards are much more complex than that.

## Example 2: Trace-elements in tephra samples 

This example is from the book [Introduction to Python in Earth Science Data Analysis](https://link.springer.com/book/10.1007/978-3-030-78055-5) by Maurizio Petrelli. The Excel file `Smith_glass_post_NYT_data.xlsx` contains the chemical concentrations of trace elements of volcanic ash (tephra) from the Campi Flegrei volcano, Italy ([Smith et al., 2011](https://doi.org/10.1016/J.QUASCIREV.2011.07.012)). Let's read the `Supp_traces` sheet from this file:

In [ ]:
path = os.path.join("..", "data", "Smith_glass_post_NYT_data.xlsx") # a safe path
my_dataset = pd.read_excel(path, sheet_name="Supp_traces")
my_dataset.head()

Not all the columns of the table are listed. To get a complete list of the columns, you can use the `list` function:

In [ ]:
list(my_dataset)

And to quickly get a descriptive statistics of the dataset, you can use the `DataFrame.describe` method:

In [ ]:
my_dataset.describe()

Let's plot the elements Zr versus Th in a scatter diagram. This is done by selecting and plotting the Zr and Th columns, as shown below:

In [ ]:
x = my_dataset.Zr # select Zr column, my_dataset["Zr"] also works
y = my_dataset.Th # select Th column, my_dataset["Th"] also works

fig, ax = plt.subplots()
ax.scatter(x, y)
ax.set_title("Zr vs. Th")
ax.set_xlabel("Zr [ppm]")
ax.set_ylabel("Th [ppm]");

Now, suppose that we want to define two datasets based on the concentration of Zr, one dataset with Zr >= 450 ppm, and another with Zr < 450 ppm. In addition we want to plot these datasets with different colors:

In [ ]:
# define two sub-datasets for Zr >= 450 and Zr < 450
my_sub_dataset1 = my_dataset[x >= 450] # sub_dataset with Zr >= 450 ppm
my_sub_dataset2 = my_dataset[x < 450] # sub_dataset with Zr >= 450 ppm

# generate the scatter Zr Vs Th diagram for Zr >= 450
# in blue, also defining the legend caption as "Zr >= 450 [ppm]"
x1 = my_sub_dataset1.Zr
y1 = my_sub_dataset1.Th
fig, ax = plt.subplots()
ax.scatter(x1, y1, color="blue", label="Zr >= 450 [ppm]")

# generate the scatter Zr Vs Th diagram for Zr < 450
# in red, also defining the legend caption as "Zr < 450 [ppm]"
x2 = my_sub_dataset2.Zr
y2 = my_sub_dataset2.Th
ax.scatter(x2, y2, color="red", label="Zr < 450 [ppm]")

ax.set_title("Zr vs Th")
ax.set_xlabel("Zr [ppm]")
ax.set_ylabel("Th [ppm]")
ax.legend();

Let's filter and plot the data by the labels in the `Epoch` column. To do this, we just make a list of epochs, and then using a `for` loop, we iterate over all epochs to filter and plot the data:

In [ ]:
epochs = ["one","two","three","three-b"] # list of epochs

fig, ax = plt.subplots()

# filter and plot the dataset by epochs
for epoch in epochs: # for each epoch
    my_data = my_dataset[my_dataset.Epoch == epoch] # data belonging to epoch
    ax.scatter(my_data.Zr, my_data.Th, label="Epoch " + epoch) # plot data belonging to epoch

ax.set_title("Zr vs Th")
ax.set_xlabel("Zr [ppm]")
ax.set_ylabel("Th [ppm]")
ax.legend();

Let's make a more fancy, publication ready plot. To do this, we define in addition colors and markers lists for the different epochs, and include those in the `for` loop. The `zip` function allows us to iterate over the three lists at the same time:

In [ ]:
# make a publication ready plot
epochs = ["one","two","three","three-b"] # list of epochs
colors = ["#c8b4ba", "#f3ddb3", "#c1cd97", "#e18d96"] # list of colors in hexadecimal format for the epochs
markers = ["o", "s", "d", "v"] # list of markers for the epochs

fig, ax = plt.subplots(figsize=(8,6))

# filter and plot the dataset by epochs
for (epoch, color, marker) in zip(epochs, colors, markers): # iterate over lists
    my_data = my_dataset[my_dataset.Epoch == epoch] # data belonging to epoch
    ax.scatter(my_data.Zr, my_data.Th, marker=marker, s=50, 
               c=color, edgecolor="0", label="Epoch " + epoch) # plot the data

ax.set_xlabel("Zr [ppm]")
ax.set_ylabel("Th [ppm]")
# ax.grid(True)
ax.legend(title = "Phlegraean Fields \n Age < 15 ky");

In the cell above, we specified the colors using hexadecimal values. These codes can be found using a hexadecimal color picker; [this one](https://g.co/kgs/hpBWRg) for example. The markers can be found from this [link](https://matplotlib.org/stable/api/markers_api.html). 

Let's save our figure:

In [ ]:
fig.savefig("ZrvsTh.pdf", dpi=200)

That's the end of this notebook. Throughout the course, we will use a lot `pandas`.

To practice, try the exercises in `day2/lab/lab2.pdf`